# Optimal Aptamer Selection

This notebook selects optimal aptamers for each target molecule based on binding affinity, specificity, and structural stability.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Add the project root to the path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
from src.models.binding_affinity import BindingAffinityPredictor
from src.models.cross_reactivity import CrossReactivityAnalyzer
from src.aptamer_selection.selector import AptamerSelector
from src.aptamer_selection.specificity_optimizer import AptamerOptimizer
from src.visualization.plot_utils import AptamerVisualizer

## Load Data and Models

In [ ]:
# Load cross-reactivity analysis results or feature-enriched data
crossreact_path = '../data/processed/cross_reactivity_analysis.csv'

if os.path.exists(crossreact_path):
    df = pd.read_csv(crossreact_path)
    print(f"Loaded cross-reactivity analysis results: {len(df)} rows")
else:
    # Try feature-enriched data
    feature_path = '../data/processed/aptamers_with_features.csv'
    if os.path.exists(feature_path):
        df = pd.read_csv(feature_path)
        print(f"Loaded feature-enriched data: {len(df)} rows")
    else:
        # Try preprocessed data
        processed_path = '../data/processed/preprocessed_aptamers.csv'
        if os.path.exists(processed_path):
            df = pd.read_csv(processed_path)
            print(f"Loaded preprocessed data: {len(df)} rows")
        else:
            df = pd.DataFrame()
            print("No data available. Please run the previous notebooks first.")

In [ ]:
# Check if required columns are present
if len(df) > 0:
    if 'Target_Name' not in df.columns:
        print("WARNING: No 'Target_Name' column found. Aptamer selection requires target information.")
    else:
        print(f"Found {df['Target_Name'].nunique()} unique targets in the dataset")
        print(df['Target_Name'].value_counts())
        
    seq_col = 'Sequence' if 'Sequence' in df.columns else 'sequence'
    if seq_col not in df.columns:
        print("WARNING: No sequence column found.")

In [ ]:
# Load previously trained models
binding_model = BindingAffinityPredictor()
binding_model_path = '../models/binding_affinity_model.pkl'

if os.path.exists(binding_model_path):
    try:
        binding_model.load_model(binding_model_path)
        print(f"Loaded binding affinity model from {binding_model_path}")
    except Exception as e:
        print(f"Error loading binding affinity model: {str(e)}")
else:
    print(f"No binding affinity model found at {binding_model_path}")

cross_reactivity_model = CrossReactivityAnalyzer()
cr_model_path = '../models/cross_reactivity_model.pkl'

if os.path.exists(cr_model_path):
    try:
        cross_reactivity_model.load_model(cr_model_path)
        print(f"Loaded cross-reactivity model from {cr_model_path}")
    except Exception as e:
        print(f"Error loading cross-reactivity model: {str(e)}")
else:
    print(f"No cross-reactivity model found at {cr_model_path}")

## Initialize Aptamer Selector

In [ ]:
# Configure and initialize the aptamer selector
selector_config = {
    'specificity_weight': 0.6,         # Weight for specificity in selection score
    'binding_affinity_weight': 0.4,    # Weight for binding affinity in selection score
    'structural_stability_weight': 0.3, # Weight for structural stability in selection score
    'min_binding_score': 0.7,          # Minimum binding score threshold
    'max_cross_reactivity': 0.2        # Maximum acceptable cross-reactivity
}

selector = AptamerSelector(selector_config)

# Set the models if they were successfully loaded
if hasattr(binding_model, 'model') and binding_model.model is not None:
    selector.binding_model = binding_model
    
if hasattr(cross_reactivity_model, 'model') and cross_reactivity_model.model is not None:
    selector.cross_reactivity_model = cross_reactivity_model

## Define Target Molecules

In [ ]:
# Define the target molecules of interest
if 'Target_Name' in df.columns:
    all_targets = df['Target_Name'].unique().tolist()
    print(f"Available targets in the dataset: {all_targets}")
else:
    # Default targets if none in the dataset
    all_targets = ['fentanyl', 'methamphetamine', 'benzodiazepine', 'xylazine', 'nitazene']
    print(f"Using default targets: {all_targets}")

# Allow user to select specific targets
targets_of_interest = all_targets
print(f"\nSelecting aptamers for targets: {targets_of_interest}")

## Select Optimal Aptamers

In [ ]:
# Select optimal aptamers for each target
if len(df) > 0:
    num_aptamers_per_target = 5  # Number of aptamers to select per target
    
    print(f"Selecting {num_aptamers_per_target} optimal aptamers for each target...")
    selected_aptamers = selector.select_optimal_aptamers(
        df, targets_of_interest, n_per_target=num_aptamers_per_target
    )
    
    print(f"\nSelected {len(selected_aptamers)} aptamers in total:")
    if 'Target_Name' in selected_aptamers.columns:
        print(selected_aptamers['Target_Name'].value_counts())

In [ ]:
# Display the selected aptamers
if 'selected_aptamers' in locals() and len(selected_aptamers) > 0:
    # Define columns to display
    display_cols = ['Target_Name', 'selection_score', 'specificity_score']
    
    # Add sequence column if available
    seq_col = 'Sequence' if 'Sequence' in selected_aptamers.columns else 'sequence'
    if seq_col in selected_aptamers.columns:
        display_cols = [seq_col] + display_cols
    
    # Add binding column if available
    binding_col = None
    for col in selected_aptamers.columns:
        if 'binding' in col.lower() or 'affinity' in col.lower():
            binding_col = col
            break
    
    if binding_col:
        display_cols.append(binding_col)
    
    # Add structural stability if available
    if 'stability_score' in selected_aptamers.columns:
        display_cols.append('stability_score')
    
    # Display the selected aptamers grouped by target
    for target in targets_of_interest:
        target_aptamers = selected_aptamers[selected_aptamers['Target_Name'] == target]
        
        if len(target_aptamers) > 0:
            print(f"\n--- Selected aptamers for {target} ---")
            display(target_aptamers[display_cols].sort_values('selection_score', ascending=False))
        else:
            print(f"\nNo aptamers selected for {target}")

## Verify Aptamer Quality

In [ ]:
# Verify the quality of selected aptamers
if 'selected_aptamers' in locals() and len(selected_aptamers) > 0:
    verified_aptamers = selector.verify_aptamer_quality(selected_aptamers, targets_of_interest)
    
    # Display the additional quality metrics
    quality_cols = ['Target_Name', 'quality_category', 'g4_potential', 'homopolymer_risk', 'estimated_tm']
    
    # Add sequence column if available
    if seq_col in verified_aptamers.columns:
        quality_cols = [seq_col] + quality_cols
    
    print("Verified aptamer quality metrics:")
    display(verified_aptamers[quality_cols])
    
    # Count aptamers by quality category
    if 'quality_category' in verified_aptamers.columns:
        quality_counts = verified_aptamers['quality_category'].value_counts()
        print("\nAptamers by quality category:")
        print(quality_counts)

## Visualize Selected Aptamers

In [ ]:
# Visualize binding affinity vs specificity for selected aptamers
if 'verified_aptamers' in locals() and len(verified_aptamers) > 0:
    # Create visualizer
    visualizer = AptamerVisualizer()
    
    # Plot binding vs specificity
    visualizer.plot_binding_vs_specificity(verified_aptamers, color_by_target=True)
    
    # Create dashboard
    visualizer.create_dashboard(verified_aptamers)

In [ ]:
# Visualize structures of a few selected aptamers
if 'verified_aptamers' in locals() and len(verified_aptamers) > 0:
    if 'predicted_structure' in verified_aptamers.columns and seq_col in verified_aptamers.columns:
        # Get the top aptamer for each target
        top_aptamers = []
        for target in targets_of_interest:
            target_aptamers = verified_aptamers[verified_aptamers['Target_Name'] == target]
            if len(target_aptamers) > 0:
                top_aptamer = target_aptamers.iloc[0]
                top_aptamers.append(top_aptamer)
        
        # Visualize the structures
        for aptamer in top_aptamers:
            visualizer.plot_structure_visualization(
                aptamer[seq_col],
                aptamer['predicted_structure'],
                title=f"Structure for {aptamer['Target_Name']} aptamer"
            )

## Compare Selected Aptamers

In [ ]:
# Compare sequence similarity between selected aptamers
if 'verified_aptamers' in locals() and len(verified_aptamers) > 0 and seq_col in verified_aptamers.columns:
    # Limit to a manageable number of sequences for visualization
    if len(verified_aptamers) > 20:
        # Take top 3-4 for each target
        sample_aptamers = []
        for target in targets_of_interest:
            target_aptamers = verified_aptamers[verified_aptamers['Target_Name'] == target]
            if len(target_aptamers) > 0:
                sample_aptamers.append(target_aptamers.head(min(4, len(target_aptamers))))
        
        sample_df = pd.concat(sample_aptamers, ignore_index=True)
    else:
        sample_df = verified_aptamers
    
    # Create sequence labels
    labels = [f"{row['Target_Name']} #{i+1}" for i, (_, row) in enumerate(sample_df.iterrows())]
    
    # Visualize sequence similarity
    visualizer.plot_sequence_similarity_heatmap(
        sample_df[seq_col].tolist(),
        labels=labels
    )

In [ ]:
# Compare features across targets
if 'verified_aptamers' in locals() and len(verified_aptamers) > 0 and 'Target_Name' in verified_aptamers.columns:
    # Select features to compare
    features_to_compare = [
        'gc_content' if 'gc_content' in verified_aptamers.columns else 'GC_Content',
        'length',
        'specificity_score',
        'stability_score' if 'stability_score' in verified_aptamers.columns else None,
        'estimated_tm' if 'estimated_tm' in verified_aptamers.columns else None
    ]
    
    # Filter out None values
    features_to_compare = [f for f in features_to_compare if f is not None and f in verified_aptamers.columns]
    
    if features_to_compare:
        # Create multi-panel plot
        fig, axes = plt.subplots(1, len(features_to_compare), figsize=(4 * len(features_to_compare), 5))
        
        for i, feature in enumerate(features_to_compare):
            if len(features_to_compare) == 1:
                ax = axes
            else:
                ax = axes[i]
                
            sns.boxplot(x='Target_Name', y=feature, data=verified_aptamers, ax=ax)
            ax.set_title(f'{feature} by Target')
            ax.set_xlabel('Target')
            ax.set_ylabel(feature)
            ax.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()

## Optimize Aptamers (Optional)

In [ ]:
# Optimize aptamers to improve specificity (optional - can be time-consuming)
run_optimization = False  # Set to True to run optimization

if run_optimization and 'verified_aptamers' in locals() and len(verified_aptamers) > 0:
    # Initialize optimizer
    optimizer = AptamerOptimizer({
        'optimization_iterations': 50,  # Higher values for better results (1000+ for production)
        'population_size': 50,          # Higher values for better results (200+ for production)
        'mutation_rate': 0.05,
        'crossover_rate': 0.8,
        'specificity_weight': 0.7,      # Emphasize specificity in optimization
        'binding_affinity_weight': 0.3
    })
    
    # Set the models
    if hasattr(binding_model, 'model') and binding_model.model is not None:
        optimizer.binding_model = binding_model
        
    if hasattr(cross_reactivity_model, 'model') and cross_reactivity_model.model is not None:
        optimizer.cross_reactivity_model = cross_reactivity_model
    
    print("Starting aptamer optimization...")
    optimized_aptamers = optimizer.run_parallel_optimization(
        verified_aptamers,
        targets=targets_of_interest,
        num_optimized=3  # Number of optimized aptamers per target
    )
    
    print(f"Optimization complete. Generated {len(optimized_aptamers)} optimized aptamers.")
    
    # Display optimized aptamers
    display_cols = ['Target_Name', 'optimization_fitness', 'specificity_score']
    
    if seq_col in optimized_aptamers.columns:
        display_cols = [seq_col] + display_cols
    
    if 'predicted_affinity' in optimized_aptamers.columns:
        display_cols.append('predicted_affinity')
    
    print("\nOptimized aptamers:")
    display(optimized_aptamers[display_cols].sort_values(['Target_Name', 'optimization_fitness'], ascending=[True, False]))
else:
    print("Skipping aptamer optimization")

## Compare Original vs Optimized Aptamers

In [ ]:
# Compare original vs optimized aptamers (if optimization was run)
if run_optimization and 'optimized_aptamers' in locals() and len(optimized_aptamers) > 0:
    # Compare specificity scores
    orig_specificity = verified_aptamers.groupby('Target_Name')['specificity_score'].mean()
    opt_specificity = optimized_aptamers.groupby('Target_Name')['specificity_score'].mean()
    
    # Calculate improvement
    targets = set(orig_specificity.index) | set(opt_specificity.index)
    
    comparison_data = []
    for target in targets:
        orig_score = orig_specificity.get(target, np.nan)
        opt_score = opt_specificity.get(target, np.nan)
        
        if not np.isnan(orig_score) and not np.isnan(opt_score):
            improvement = (opt_score - orig_score) / orig_score * 100
        else:
            improvement = np.nan
        
        comparison_data.append({
            'Target': target,
            'Original_Specificity': orig_score,
            'Optimized_Specificity': opt_score,
            'Improvement_Percent': improvement
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print("Specificity improvement from optimization:")
    display(comparison_df)
    
    # Visualize the improvement
    plt.figure(figsize=(10, 6))
    x = range(len(comparison_df))
    width = 0.35
    
    plt.bar([i - width/2 for i in x], comparison_df['Original_Specificity'], width, label='Original')
    plt.bar([i + width/2 for i in x], comparison_df['Optimized_Specificity'], width, label='Optimized')
    
    plt.xlabel('Target')
    plt.ylabel('Average Specificity Score')
    plt.title('Specificity Improvement from Optimization')
    plt.xticks(x, comparison_df['Target'])
    plt.legend()
    plt.tight_layout()
    plt.show()

## Save Final Results

In [ ]:
# Save selected aptamers
if 'verified_aptamers' in locals() and len(verified_aptamers) > 0:
    output_dir = '../results'
    os.makedirs(output_dir, exist_ok=True)
    
    # Save selected aptamers
    selected_path = os.path.join(output_dir, 'selected_aptamers.csv')
    verified_aptamers.to_csv(selected_path, index=False)
    print(f"Selected aptamers saved to {selected_path}")
    
    # Save optimized aptamers if available
    if run_optimization and 'optimized_aptamers' in locals() and len(optimized_aptamers) > 0:
        optimized_path = os.path.join(output_dir, 'optimized_aptamers.csv')
        optimized_aptamers.to_csv(optimized_path, index=False)
        print(f"Optimized aptamers saved to {optimized_path}")

In [ ]:
# Create a summary report of the final aptamers
if 'verified_aptamers' in locals() and len(verified_aptamers) > 0:
    # Create a report DataFrame
    report_data = []
    
    for target in targets_of_interest:
        target_aptamers = verified_aptamers[verified_aptamers['Target_Name'] == target]
        
        if len(target_aptamers) > 0:
            # Get top 3 aptamers
            top_aptamers = target_aptamers.sort_values('selection_score', ascending=False).head(3)
            
            for i, (_, aptamer) in enumerate(top_aptamers.iterrows(), 1):
                aptamer_seq = aptamer[seq_col] if seq_col in aptamer else "N/A"
                selection_score = aptamer.get('selection_score', "N/A")
                specificity = aptamer.get('specificity_score', "N/A")
                binding = aptamer.get('predicted_affinity', aptamer.get('binding_affinity', "N/A"))
                stability = aptamer.get('stability_score', "N/A")
                quality = aptamer.get('quality_category', "N/A")
                
                # Add to report
                report_data.append({
                    'Target': target,
                    'Rank': i,
                    'Aptamer_Sequence': aptamer_seq,
                    'Selection_Score': selection_score,
                    'Specificity': specificity,
                    'Binding_Affinity': binding,
                    'Structural_Stability': stability,
                    'Quality': quality
                })
    
    report_df = pd.DataFrame(report_data)
    
    # Save the report
    report_path = os.path.join(output_dir, 'aptamer_selection_report.csv')
    report_df.to_csv(report_path, index=False)
    print(f"Selection report saved to {report_path}")
    
    # Display the report
    print("\nAptamer Selection Report:")
    display(report_df)

## Final Selected Aptamer Sequences

In [ ]:
# Display the final selected aptamer sequences for each target
if 'verified_aptamers' in locals() and len(verified_aptamers) > 0 and seq_col in verified_aptamers.columns:
    print("Final Selected Aptamer Sequences:\n")
    
    for target in targets_of_interest:
        target_aptamers = verified_aptamers[verified_aptamers['Target_Name'] == target]
        
        if len(target_aptamers) > 0:
            # Sort by selection score
            top_aptamers = target_aptamers.sort_values('selection_score', ascending=False)
            
            print(f"--- {target.upper()} ---")
            for i, (_, aptamer) in enumerate(top_aptamers.iterrows(), 1):
                print(f"Aptamer #{i}: {aptamer[seq_col]}")
                if 'selection_score' in aptamer:
                    print(f"   Selection Score: {aptamer['selection_score']:.4f}")
                if 'specificity_score' in aptamer:
                    print(f"   Specificity: {aptamer['specificity_score']:.4f}")
                if 'quality_category' in aptamer:
                    print(f"   Quality: {aptamer['quality_category']}")
                print()
            print()

## Conclusions

Based on our selection process, we have identified optimal aptamers for each of our target adulterants:

1. **Fentanyl**: [Fill in after running the notebook]

2. **Methamphetamine**: [Fill in after running the notebook]

3. **Benzodiazepine**: [Fill in after running the notebook]

4. **Xylazine**: [Fill in after running the notebook]

5. **Nitazene**: [Fill in after running the notebook]

These aptamers were selected based on a balanced consideration of binding affinity, specificity (minimal cross-reactivity), and structural stability, making them promising candidates for use in our test strips.